## 01. PyTorch Workflow Fundamentals
The essence of machine learning and deep learning is to take some data from the past, build an algorithm (like a neural network) to discover patterns in it and use the discoverd patterns to predict the future.

There are many ways to do this and many new ways are being discovered all the time.

But let's start small.

How about we start with a straight line?

And we see if we can build a PyTorch model that learns the pattern of the straight line and matches it.

What we're going to cover
In this module we're going to cover a standard PyTorch workflow (it can be chopped and changed as necessary but it covers the main outline of steps).

<img src="pix/01_a_pytorch_workflow.png" alt="01_a_pytorch_workflow.png">

For now, we'll use this workflow to predict a simple straight line but the workflow steps can be repeated and changed depending on the problem you're working on.

Specifically, we're going to cover:

| Topic | Contents |
| :--------------------- | :-------------------------------------------------------------- |
| 1. Getting data ready | Data can be almost anything but to get started we are going to create a simple straight line |
| 2. Building a model | Here we'll create a model to learn patterns in the data, we'll also choose a loss function, optimizer and build a training loop. |
| 3. Fitting the model to the data (training) | We have got data and a model, now let's let the model (try to) find patterns in the training data. |
| 4. Making predictions and evaluating a model (Inference) | Our MOdel's found patterns in the data, let's compare its findings to the actual (testing) data. |
| 5. Saving and loading a model | You may want to use your model elsewhere, or comeback to it later, here we'll cover that. |
| 6. Putting it all together | let's take all of the above and combine it.

In [ ]:
import torch
import math
# this ensures that the current MacOS version is at least 12.3+
print(torch.backends.mps.is_available())
# this ensures that the current current PyTorch installation was built with MPS activated.
print(torch.backends.mps.is_built())


In [1]:
import torch
from torch import nn
import matplotlib.pyplot as plt

# check pytorch version
torch.__version__


'2.0.1'

In [2]:
# set device to mps

device= 'mps' if torch.backends.mps.is_available() else 'cpu'
print(f"Using device: {device}")

Using device: mps


In [6]:
# create weights and bias
weight= 0.7
bias= 0.3

# create range values
START= 0
STOP= 1
STEP= 0.02

# create X and y (features and label)
X= torch.arange(START, STOP, STEP).unsqueeze(dim=1)
y= weight * X + bias

X[:5], y[:5]

(tensor([[0.0000],
         [0.0200],
         [0.0400],
         [0.0600],
         [0.0800]]),
 tensor([[0.3000],
         [0.3140],
         [0.3280],
         [0.3420],
         [0.3560]]))

In [8]:
# split the data in train and test sets

train_split= int(0.8*len(X))

X_train= X[:train_split]
X_test= X[train_split:]

y_train= y[:train_split]
y_test= y[train_split:]

len(X_train), len(y_train), len(X_test), len(y_test)

(40, 40, 10, 10)

In [19]:
# Building a torch model

# subclass nn.module to build our model
class LinearRegressionModelV2(nn.Module):

    def __init__(self):
        super().__init__()

        # use nn.linear to create model parameters
        self.linear_layer= nn.Linear(in_features=1, 
                                    out_features=1)

    # define the forward computation (input data x flows through the nn.Linear
    def forward(self, x: torch.Tensor) -> torch.Tensor:
        return self.linear_layer(x)


# set the manual random seed when creating the model
torch.manual_seed(42)
model_1= LinearRegressionModelV2()
model_1, model_1.state_dict()
    

(LinearRegressionModelV2(
   (linear_layer): Linear(in_features=1, out_features=1, bias=True)
 ),
 OrderedDict([('linear_layer.weight', tensor([[0.7645]])),
              ('linear_layer.bias', tensor([0.8300]))]))

In [13]:
# check model device
next(model_1.parameters()).device

device(type='cpu')

In [15]:
# Set model to GPU if available
model_1.to(device)
next(model_1.parameters()).device

device(type='mps', index=0)

### Training

In [26]:
# create a loss function
loss_fn= nn.L1Loss()

# create an optimizer
optimizer= torch.optim.SGD(params= model_1.parameters(),
                           lr= 0.01)

In [27]:
torch.manual_seed(42)

# set the number of epochs
epochs= 1000

# put all data on GPU, an error will occur if this is not done
X_train.to(device)
X_test.to(device)
y_train.to(device)
y_test.to(device)


# begin the training loop
for epochs in range(epochs):
    # training
    model_1.train()

    # forward pass
    y_pred= model_1(X_train)

    # calculate the loss;
    loss= loss_fn(y_pred, y_train)

    # zero grad optimizer
    optimizer.zero_grad()

    # 4. loss backwards
    loss.backward()

    #step the optimizer
    optimizer.step()

    # testing
    # forward pass
    with torch.inference_mode():
        test_pred= model_1(X_test)

        # calculate the loss
        test_loss= loss_fn(test_pred, y_test)

    if epochs% 100 == 0:
        print(f"Epoch: {epochs} | train loss: {loss} | test loss: {test_loss}")

Epoch: 0 | train loss: 0.05761389806866646 | test loss: 0.004697275348007679
Epoch: 100 | train loss: 0.004472536034882069 | test loss: 0.005817919969558716
Epoch: 200 | train loss: 0.004472536034882069 | test loss: 0.005817919969558716
Epoch: 300 | train loss: 0.004472536034882069 | test loss: 0.005817919969558716
Epoch: 400 | train loss: 0.004472536034882069 | test loss: 0.005817919969558716
Epoch: 500 | train loss: 0.004472536034882069 | test loss: 0.005817919969558716
Epoch: 600 | train loss: 0.004472536034882069 | test loss: 0.005817919969558716
Epoch: 700 | train loss: 0.004472536034882069 | test loss: 0.005817919969558716
Epoch: 800 | train loss: 0.004472536034882069 | test loss: 0.005817919969558716
Epoch: 900 | train loss: 0.004472536034882069 | test loss: 0.005817919969558716


In [28]:
model_1, model_1.state_dict()


(LinearRegressionModelV2(
   (linear_layer): Linear(in_features=1, out_features=1, bias=True)
 ),
 OrderedDict([('linear_layer.weight', tensor([[0.6936]])),
              ('linear_layer.bias', tensor([0.2980]))]))

In [30]:
import sklearn
from sklearn.datasets import make_circles

In [1]:
!pip install pytorch-ignite


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 266.8/266.8 kB 592.3 kB/s eta 0:00:00a 0:00:01
